# Joshua Kobuskie
## CS670-850
## June 22nd, 2025
## Project 1 Part 1

The Lion, Goat, and Grass problem is a classic example of a river crossing puzzle. In this problem, a farmer needs to transport a lion, a goat, and grass across a river. The farmer has a boat that is small and can only carry one of these items along with the farmer at a time. The challenge arises from the following constraints:  
1. If the lion and the goat are left together on the same side of the river without the farmer, the lion will eat the goat.  
2. If the goat and the grass are left together on the same side of the river without the farmer, the goat will eat the grass.  

The goal is to devise a strategy that allows the farmer to transport all three items (the lion, the goat, and the grass) across the river without any of them being eaten.  

We can represent the state with a tuple of four boolean values (farmer, lion, goat, grass), where False might represent an item being on the original side (left) and True being on the destination side (right). Not all tuples are valid.

A tuple is defined to capture the state of the man, lion, goat, and grass in this order, with "False" meaning that the item is on the left side and "True" meaning that the item is on the right side. The index for each item is saved for easier reference. The check_constraints function is then defined to determine if a given tuple will result in items being eaten, which would make the state invalid. The print_state function is defined to show which items are on each side.

In [ ]:
# The tuple will store the information as follows:
# (Man, Lion, Goat, Grass)
# False will mean that the item is on the left side
# True will mean that the item is on the right side

# Easier to remember indexes
man = 0
lion = 1
goat = 2
grass = 3

# Enforce Constraints
def validate_problem_constraints(problem_configuration):

    # Lion eats goat if without the farmer
    if problem_configuration[lion] == problem_configuration[goat] and problem_configuration[man] != problem_configuration[lion]:
        return False

    # Goat eats grass if without the farmer
    if problem_configuration[goat] == problem_configuration[grass] and problem_configuration[man] != problem_configuration[goat]:
        return False
    
    return True

# Print state of items
def display_current_problem_layout(problem_configuration):
    print("Starting Side: {}{}{}{}".format("Man " if not problem_configuration[man] else "", "Lion " if not problem_configuration[lion] else "", "Goat " if not problem_configuration[goat] else "", "Grass " if not problem_configuration[grass] else ""))
    print("Target Side: {}{}{}{}".format("Man " if problem_configuration[man] else "", "Lion " if problem_configuration[lion] else "", "Goat " if problem_configuration[goat] else "", "Grass " if problem_configuration[grass] else ""))
    print()

A quick check is performed to confirm that the check_constraints function and print_state function work as intended.

In [ ]:
sample_case_1 = (False, False, False, False)
sample_case_2 = (False, True, True, False)
sample_case_3 = (True, True, False, False)

print("Valid: Everyone is on the left side: {}".format(validate_problem_constraints(sample_case_1)))
display_current_problem_layout(sample_case_1)
print()

print("Invalid: The lion and goat are left without the farmer: {}".format(validate_problem_constraints(sample_case_2)))
display_current_problem_layout(sample_case_2)
print()

print("Invalid: The goat and grass are left without the farmer: {}".format(validate_problem_constraints(sample_case_3)))
display_current_problem_layout(sample_case_3)

Valid: Everyone is on the left side: True
Starting Side: Man Lion Goat Grass 
Target Side: 


Invalid: The lion and goat are left without the farmer: False
Starting Side: Man Grass 
Target Side: Lion Goat 


Invalid: The goat and grass are left without the farmer: False
Starting Side: Goat Grass 
Target Side: Man Lion 



The starting state and goal states are then defined, and the next_states function is created to determine which moves can be made. This function checks each combination of moves with the constraints of the problem using the check_constraints function, and also considers if the man is on the same side as the item that he is attempting to move. The next_states function returns the possible next states from the given state that are within the constraints.

In [ ]:
initial_configuration = (False, False, False, False)
target_configuration = (True, True, True, True)

def generate_potential_children_configurations(problem_configuration):
    potential_children_configurations = []

    # Man can always move
    potential_state = (not problem_configuration[man], problem_configuration[lion], problem_configuration[goat], problem_configuration[grass])
    # Check if we can move without violating the constraints
    if validate_problem_constraints(potential_state):
        potential_children_configurations.append(potential_state)

    # Check if man and lion are on the same side
    if problem_configuration[man] == problem_configuration[lion]:
        potential_state = (not problem_configuration[man], not problem_configuration[lion], problem_configuration[goat], problem_configuration[grass])
        # Check if we can move without violating the constraints
        if validate_problem_constraints(potential_state):
            potential_children_configurations.append(potential_state)
        
    # Check if man and goat are on the same side
    if problem_configuration[man] == problem_configuration[goat]:
        potential_state = (not problem_configuration[man], problem_configuration[lion], not problem_configuration[goat], problem_configuration[grass])
        # Check if we can move without violating the constraints
        if validate_problem_constraints(potential_state):
            potential_children_configurations.append(potential_state)
        
    # Check if man and grass are on the same side
    if problem_configuration[man] == problem_configuration[grass]:
        potential_state = (not problem_configuration[man], problem_configuration[lion], problem_configuration[goat], not problem_configuration[grass])
        # Check if we can move without violating the constraints
        if validate_problem_constraints(potential_state):
            potential_children_configurations.append(potential_state)
    
    return potential_children_configurations

The Breadth First Search Algorithm is defined using a First-In First-Out (FIFO) queue. Each item in the queue stores the path to the current state, and the final value in the path is the current state. This allows for easy tracking of the path to the goal state when reached. Each item is popped off the front of the queue to ensure a FIFO structure and the state is compared against the goal state. If the state matches the goal state, the path to the goal state is returned. If it is not the goal state, the possible next states from this state are determined using the next_states function and only unvisited states will be explored to make the search more efficient. Unvisited next states are then added to the back of the queue, and the queue is searched until either the goal state is found or there is no possible solution discovered.

In [ ]:
def breadth_first_search(initial_configuration, target_configuration):

    # Storing each state and the path of states that follow as an array in the queue
    # BFS uses a FIFO Queue
    frontier_queue = [[initial_configuration]]
    explored_configurations = set([initial_configuration])

    while frontier_queue:
        # Pop left for FIFO
        moves_sequence = frontier_queue.pop(0)
        # Current state in this path
        problem_configuration = moves_sequence[-1]

        # Reached the goal state
        if problem_configuration == target_configuration:
            return moves_sequence

        # Check all possible moves
        for next_problem_configuration in generate_potential_children_configurations(problem_configuration):
            # Only evaluate if we have not seen this state before and then add to queue
            if next_problem_configuration not in explored_configurations:
                explored_configurations.add(next_problem_configuration)
                next_moves_sequence = moves_sequence.copy()
                next_moves_sequence.append(next_problem_configuration)
                frontier_queue.append(next_moves_sequence)
    
    # No solution
    return None

The BFS algorithm is used to find the solution to the lion, goat, grass problem and the results are printed.

In [ ]:
final_moves_sequence = breadth_first_search(initial_configuration, target_configuration)

if final_moves_sequence:
    for problem_configuration in final_moves_sequence:
        display_current_problem_layout(problem_configuration)
else:
    print("No solution!")

Starting Side: Man Lion Goat Grass 
Target Side: 

Starting Side: Lion Grass 
Target Side: Man Goat 

Starting Side: Man Lion Grass 
Target Side: Goat 

Starting Side: Grass 
Target Side: Man Lion Goat 

Starting Side: Man Goat Grass 
Target Side: Lion 

Starting Side: Goat 
Target Side: Man Lion Grass 

Starting Side: Man Goat 
Target Side: Lion Grass 

Starting Side: 
Target Side: Man Lion Goat Grass 

